# Clean Notebook for once hopefully

In [ ]:
import os

import torch
from pytorch_lightning import seed_everything

from bci_aic3.preprocess import preprocess_and_save
from bci_aic3.data import load_raw_data
from bci_aic3.config import load_model_config, load_processing_config
from bci_aic3.paths import (
    LABEL_MAPPING_PATH,
    MI_CONFIG_PATH,
    MI_RUNS_DIR,
    RAW_DATA_DIR,
    SSVEP_CONFIG_PATH,
    SSVEP_RUNS_DIR,
)
from bci_aic3.util import read_json_to_dict
from bci_aic3.train import train_and_save

In [ ]:
# To utilize cuda cores 
torch.set_float32_matmul_precision("medium")

# Code necessary to create reproducible runs
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
seed_everything(42, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)

In [ ]:
task_type = "MI"

In [ ]:
config_path = None
save_path = None
if task_type.upper() == "MI":
    config_path = MI_CONFIG_PATH
    save_path = MI_RUNS_DIR
elif task_type.upper() == "SSVEP":
    config_path = SSVEP_CONFIG_PATH
    save_path = SSVEP_RUNS_DIR
else:
    raise (
        ValueError(f"Invalid task_type: {task_type}.\nValid task_type (MI) or (SSVEP)")
    )


processing_config = load_processing_config(config_path)
label_mapping = read_json_to_dict(LABEL_MAPPING_PATH)

train, val, _ = load_raw_data(
    base_path=RAW_DATA_DIR,
    task_type=task_type,
    label_mapping=label_mapping,
)

In [ ]:
preprocess_and_save(train_dataset=train,
                    val_dataset=val,
                    task_type=task_type,
                    processing_config=processing_config)

In [ ]:
train_and_save(task_type=task_type)